In [3]:
# 1) Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

# 2) Set the ZIP path (Option A: explicit path)
zip_path = "/content/drive/MyDrive/Yelp-JSON.zip"

# If you're not sure where it is, Option B: search for it in MyDrive
# (Uncomment this block if needed)
# import os
# matches = []
# for root, _, files in os.walk("/content/drive/MyDrive"):
#     if "Yelp-JSON.zip" in files:
#         matches.append(os.path.join(root, "Yelp-JSON.zip"))
# if not matches:
#     raise FileNotFoundError("Could not find Yelp-JSON.zip anywhere in /MyDrive")
# zip_path = matches[0]
# print("Found ZIP at:", zip_path)

# 3) Choose an output folder in Drive (this will show up in Google Drive)
import os
out_dir = "/content/drive/MyDrive/Yelp-JSON-Result"
os.makedirs(out_dir, exist_ok=True)

# 4) Unzip
import zipfile
with zipfile.ZipFile(zip_path, 'r') as z:
    z.extractall(out_dir)

print("✅ Unzipped to:", out_dir)

# 5) Show extracted contents (top-level)
print("\nTop-level files/folders:")
for name in sorted(os.listdir(out_dir))[:50]:
    print(" -", name)

# (Optional) Show total file count
total = 0
for _, _, files in os.walk(out_dir):
    total += len(files)
print(f"\nTotal files extracted: {total}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Unzipped to: /content/drive/MyDrive/Yelp-JSON-Result

Top-level files/folders:
 - Yelp JSON
 - __MACOSX

Total files extracted: 4


In [9]:
from google.colab import drive
drive.mount("/content/drive")

# CHANGE this path to where the tar actually is in your Drive
tar_path = "/content/drive/MyDrive/yelp_dataset.tar"   # or .tar.gz / .tgz
out_dir  = "/content/drive/MyDrive/extracted_folder"

import os, tarfile
os.makedirs(out_dir, exist_ok=True)

with tarfile.open(tar_path, "r:*") as tar:
    tar.extractall(path=out_dir)

print("✅ Extracted to:", out_dir)
!ls -lah "$out_dir" | head


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


/tmp/ipython-input-1888330997.py:12: DeprecationWarning: Python 3.14 will, by default, filter extracted tar archives and reject files or modify their metadata. Use the filter argument to control this behavior.
  tar.extractall(path=out_dir)


✅ Extracted to: /content/drive/MyDrive/extracted_folder
total 8.7G
-rw------- 1 root root  79K Feb 15  2022 Dataset_User_Agreement.pdf
-rw------- 1 root root 114M Jan 19  2022 yelp_academic_dataset_business.json
-rw------- 1 root root 274M Jan 19  2022 yelp_academic_dataset_checkin.json
-rw------- 1 root root 5.0G Jan 19  2022 yelp_academic_dataset_review.json
-rw------- 1 root root 173M Jan 19  2022 yelp_academic_dataset_tip.json
-rw------- 1 root root 3.2G Jan 19  2022 yelp_academic_dataset_user.json


In [2]:
import os

from google.colab import drive
drive.mount("/content/drive")

# Try both common "MyDrive" vs "My Drive" spellings
base_candidates = [
    "/content/drive/MyDrive/extracted_folder",
    "/content/drive/My Drive/extracted_folder",
]

DATA_DIR = None
for c in base_candidates:
    if os.path.isdir(c):
        DATA_DIR = c
        break

print("Detected DATA_DIR:", DATA_DIR)

if DATA_DIR is None:
    # show what is inside /content/drive to help you spot the real path
    print("\nFolders at /content/drive:")
    print(os.listdir("/content/drive"))
    raise FileNotFoundError("Could not find extracted_folder under MyDrive or My Drive.")

# IMPORTANT: use the filenames shown in your screenshot
BUSINESS_PATH = os.path.join(DATA_DIR, "yelp_academic_dataset_business.json")
REVIEW_PATH   = os.path.join(DATA_DIR, "yelp_academic_dataset_review.json")
USER_PATH     = os.path.join(DATA_DIR, "yelp_academic_dataset_user.json")

print("\nChecking files:")
for p in [BUSINESS_PATH, REVIEW_PATH, USER_PATH]:
    print(p, "->", os.path.exists(p))

missing = [p for p in [BUSINESS_PATH, REVIEW_PATH, USER_PATH] if not os.path.exists(p)]
if missing:
    print("\nFiles in DATA_DIR are actually:")
    print(os.listdir(DATA_DIR)[:50])
    raise FileNotFoundError("Missing expected JSON file(s):\n" + "\n".join(missing))

print("\n✅ Paths are correct. Continue to the next cell.")


Mounted at /content/drive
Detected DATA_DIR: /content/drive/MyDrive/extracted_folder

Checking files:
/content/drive/MyDrive/extracted_folder/yelp_academic_dataset_business.json -> True
/content/drive/MyDrive/extracted_folder/yelp_academic_dataset_review.json -> True
/content/drive/MyDrive/extracted_folder/yelp_academic_dataset_user.json -> True

✅ Paths are correct. Continue to the next cell.


In [9]:
# Full-Column Yelp Pipeline: Business + Review + User (Parquet) + UI Bundle (CSV)
# INPUT (Drive):  /content/drive/MyDrive/extracted_folder/
# OUTPUT (Drive): /content/drive/MyDrive/irvine_recommender_outputs/

import os, gc
import pandas as pd
from tqdm.auto import tqdm

# ----------------------------
# 0) Mount Drive + Paths
# ----------------------------
from google.colab import drive
drive.mount("/content/drive")

IN_DIR  = "/content/drive/MyDrive/extracted_folder"
OUT_DIR = "/content/drive/MyDrive/irvine_recommender_outputs"
os.makedirs(OUT_DIR, exist_ok=True)

BUSINESS_PATH = os.path.join(IN_DIR, "yelp_academic_dataset_business.json")
REVIEW_PATH   = os.path.join(IN_DIR, "yelp_academic_dataset_review.json")
USER_PATH     = os.path.join(IN_DIR, "yelp_academic_dataset_user.json")

OUT_BUSINESS_PARQUET = os.path.join(OUT_DIR, "oc_business.parquet")
OUT_REVIEW_PARQUET   = os.path.join(OUT_DIR, "oc_review.parquet")
OUT_USER_PARQUET     = os.path.join(OUT_DIR, "oc_user.parquet")
OUT_UI_BUNDLE        = os.path.join(OUT_DIR, "ui_bundle.csv")  # not "irvine" since your data isn't Irvine

# Sanity check
if not os.path.isdir(IN_DIR):
    raise FileNotFoundError(f"Input folder not found: {IN_DIR}")
missing = [p for p in [BUSINESS_PATH, REVIEW_PATH, USER_PATH] if not os.path.exists(p)]
if missing:
    print("Files in IN_DIR:", os.listdir(IN_DIR)[:50])
    raise FileNotFoundError("Missing expected Yelp JSON file(s):\n" + "\n".join(missing))

print("✅ Input folder:", IN_DIR)
print("✅ Output folder:", OUT_DIR)

# ----------------------------
# 0.5) Ensure Parquet engine
# ----------------------------
try:
    import pyarrow as pa
    import pyarrow.parquet as pq
except Exception:
    !pip -q install pyarrow
    import pyarrow as pa
    import pyarrow.parquet as pq

# ----------------------------
# Config (your dataset's cities)
# ----------------------------
OC_CITIES = {"Santa Barbara", "Goleta", "Carpinteria", "Isla Vista", "Montecito", "Summerland"}
TARGET_UI_CITY = "Santa Barbara"  # change this if you want another city

INCLUDE_KEYWORDS = [
    "restaurants", "restaurant",
    "cafe", "cafes",
    "coffee & tea",
    "breakfast & brunch",
    "bakeries",
    "desserts",
    "sandwiches",
    "pizza",
    "bars",
    "food delivery services",
    "ice cream", "frozen yogurt",
    "seafood",
    "steakhouses",
    "mexican", "italian", "chinese", "japanese", "korean", "thai", "vietnamese",
    "american", "bbq", "burgers"
]

EXCLUDE_KEYWORDS = [
    "lawyers", "law", "attorneys",
    "dentists", "dental",
    "doctors", "health & medical", "medical",
    "automotive", "car dealers", "windshield", "auto glass",
    "home services", "plumbing", "hvac", "heating",
    "real estate",
    "insurance",
    "contractors",
    "massage", "beauty & spas",
    "shopping"
]

include_pat = r"(" + "|".join([k.replace("&", r"\&") for k in INCLUDE_KEYWORDS]) + r")"
exclude_pat = r"(" + "|".join([k.replace("&", r"\&") for k in EXCLUDE_KEYWORDS]) + r")"

# ----------------------------
# Helpers
# ----------------------------
def compute_day_type(dt_series):
    wd = dt_series.dt.weekday  # Mon=0 ... Sun=6
    out = (wd >= 5).map({True: "Weekend", False: "Weekday"}).astype("string")
    return pd.Series(pd.Categorical(out, categories=["Weekday", "Weekend"]), index=dt_series.index)

def compute_time_bucket(_dt_series):
    # Review timestamps exist in your sample ("2018-07-07 22:09:11"), but Yelp review.json sometimes varies.
    # This keeps it stable even if parse fails.
    out = pd.Series("Unknown", index=_dt_series.index, dtype="string")
    return pd.Series(pd.Categorical(out, categories=["Morning", "Lunch", "Dinner", "Late Night", "Unknown"]),
                     index=_dt_series.index)

# ----------------------------
# 1) Business table (KEEP ALL COLUMNS)
# ----------------------------
print("\n1) Loading business JSON (all columns)...")
biz = pd.read_json(BUSINESS_PATH, lines=True)

# Normalize for filtering (keep original columns too)
biz["state"] = biz["state"].astype("string").str.strip()
biz["city_norm"] = biz["city"].astype("string").str.strip().str.lower()
biz["categories"] = biz["categories"].astype("string").fillna("")

biz_ca = biz[biz["state"] == "CA"].copy()
print("CA businesses:", len(biz_ca))

oc_cities_norm = {c.lower() for c in OC_CITIES}
mask_city = biz_ca["city_norm"].isin(oc_cities_norm)

mask_foodish = biz_ca["categories"].str.lower().str.contains(include_pat, regex=True)
mask_not_nonfood = ~biz_ca["categories"].str.lower().str.contains(exclude_pat, regex=True)

biz_full = biz_ca.loc[mask_city & mask_foodish & mask_not_nonfood].copy()

print("Businesses after filter:", len(biz_full))
if len(biz_full) == 0:
    print("Top CA cities:\n", biz_ca["city"].value_counts().head(30))
    raise ValueError("Business filter returned 0 rows. Adjust cities or include/exclude keywords.")

# Drop helper col(s) you don’t want persisted
biz_full.drop(columns=["city_norm"], inplace=True, errors="ignore")

# Save ALL columns
biz_full.to_parquet(OUT_BUSINESS_PARQUET, index=False)
print(f"✅ Saved business (all cols): {OUT_BUSINESS_PARQUET} (rows={len(biz_full):,}, cols={biz_full.shape[1]})")

# Business IDs for review filtering
business_ids = set(biz_full["business_id"].astype(str).tolist())

# UI bundle (top 100 in TARGET_UI_CITY)
city_mask = biz_full["city"].astype("string").str.strip().str.lower() == TARGET_UI_CITY.lower()
ui_bundle = (
    biz_full[city_mask]
    .sort_values(["review_count", "stars"], ascending=[False, False])
    .head(100)
    .copy()
)

ui_bundle.to_csv(OUT_UI_BUNDLE, index=False)
print(f"✅ Saved UI bundle: {OUT_UI_BUNDLE} (rows={len(ui_bundle):,})")

del biz, biz_ca
gc.collect()

# ----------------------------
# 2) Review table (KEEP ALL COLUMNS + add engineered)
# ----------------------------
print("\n2) Streaming review JSON in chunks (100k) (all columns)...")
REVIEW_CHUNKSIZE = 100_000

review_writer = None
kept_reviews = 0
seen_user_ids = set()

pbar = tqdm(total=None, desc="Reviews processed", unit="rows")

for chunk in pd.read_json(REVIEW_PATH, lines=True, chunksize=REVIEW_CHUNKSIZE):
    pbar.update(len(chunk))

    chunk["business_id"] = chunk["business_id"].astype(str)
    sub = chunk[chunk["business_id"].isin(business_ids)].copy()
    if sub.empty:
        del chunk, sub
        gc.collect()
        continue

    # Parse date (your sample has timestamp)
    dt = pd.to_datetime(sub["date"], errors="coerce")
    sub["day_type"] = compute_day_type(dt)
    sub["time_bucket"] = compute_time_bucket(dt)

    seen_user_ids.update(sub["user_id"].astype(str).tolist())

    # Write ALL columns (including engineered)
    table = pa.Table.from_pandas(sub, preserve_index=False)
    if review_writer is None:
        review_writer = pq.ParquetWriter(OUT_REVIEW_PARQUET, table.schema, compression="snappy")
    review_writer.write_table(table)

    kept_reviews += len(sub)

    del chunk, sub, table, dt
    gc.collect()

pbar.close()
if review_writer is not None:
    review_writer.close()

# Ensure file exists even if empty
if kept_reviews == 0:
    pd.DataFrame().to_parquet(OUT_REVIEW_PARQUET, index=False)

print(f"✅ Saved reviews (all cols): {OUT_REVIEW_PARQUET} (rows={kept_reviews:,})")
print(f"Unique users in filtered reviews: {len(seen_user_ids):,}")

# ----------------------------
# 3) User table (KEEP ALL COLUMNS)
# ----------------------------
print("\n3) Streaming user JSON in chunks (100k) (all columns)...")
USER_CHUNKSIZE = 100_000

user_writer = None
kept_users = 0

pbar_u = tqdm(total=None, desc="Users processed", unit="rows")

for uchunk in pd.read_json(USER_PATH, lines=True, chunksize=USER_CHUNKSIZE):
    pbar_u.update(len(uchunk))

    uchunk["user_id"] = uchunk["user_id"].astype(str)
    ukeep = uchunk[uchunk["user_id"].isin(seen_user_ids)].copy()
    if ukeep.empty:
        del uchunk, ukeep
        gc.collect()
        continue

    table = pa.Table.from_pandas(ukeep, preserve_index=False)
    if user_writer is None:
        user_writer = pq.ParquetWriter(OUT_USER_PARQUET, table.schema, compression="snappy")
    user_writer.write_table(table)

    kept_users += len(ukeep)

    del uchunk, ukeep, table
    gc.collect()

pbar_u.close()
if user_writer is not None:
    user_writer.close()

# Ensure file exists even if empty
if kept_users == 0:
    pd.DataFrame().to_parquet(OUT_USER_PARQUET, index=False)

print(f"✅ Saved users (all cols): {OUT_USER_PARQUET} (rows={kept_users:,})")

# ----------------------------
# Cleanup + Summary
# ----------------------------
del business_ids, seen_user_ids, biz_full, ui_bundle
gc.collect()

print("\n✅ DONE. Outputs saved to Drive folder:")
print(" ", OUT_DIR)
print(" -", OUT_BUSINESS_PARQUET)
print(" -", OUT_REVIEW_PARQUET)
print(" -", OUT_USER_PARQUET)
print(" -", OUT_UI_BUNDLE)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Input folder: /content/drive/MyDrive/extracted_folder
✅ Output folder: /content/drive/MyDrive/irvine_recommender_outputs

1) Loading business JSON (all columns)...
CA businesses: 5203
Businesses after filter: 1349
✅ Saved business (all cols): /content/drive/MyDrive/irvine_recommender_outputs/oc_business.parquet (rows=1,349, cols=14)
✅ Saved UI bundle: /content/drive/MyDrive/irvine_recommender_outputs/ui_bundle.csv (rows=100)


/tmp/ipython-input-4063452994.py:122: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_foodish = biz_ca["categories"].str.lower().str.contains(include_pat, regex=True)
/tmp/ipython-input-4063452994.py:123: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask_not_nonfood = ~biz_ca["categories"].str.lower().str.contains(exclude_pat, regex=True)



2) Streaming review JSON in chunks (100k) (all columns)...


Reviews processed: 0rows [00:00, ?rows/s]

✅ Saved reviews (all cols): /content/drive/MyDrive/irvine_recommender_outputs/oc_review.parquet (rows=225,242)
Unique users in filtered reviews: 109,354

3) Streaming user JSON in chunks (100k) (all columns)...


Users processed: 0rows [00:00, ?rows/s]

✅ Saved users (all cols): /content/drive/MyDrive/irvine_recommender_outputs/oc_user.parquet (rows=109,354)

✅ DONE. Outputs saved to Drive folder:
  /content/drive/MyDrive/irvine_recommender_outputs
 - /content/drive/MyDrive/irvine_recommender_outputs/oc_business.parquet
 - /content/drive/MyDrive/irvine_recommender_outputs/oc_review.parquet
 - /content/drive/MyDrive/irvine_recommender_outputs/oc_user.parquet
 - /content/drive/MyDrive/irvine_recommender_outputs/ui_bundle.csv


In [4]:
import os
print(os.listdir("/content/drive"))
print(os.listdir("/content/drive/MyDrive") if os.path.exists("/content/drive/MyDrive") else "No MyDrive")
print(os.listdir("/content/drive/My Drive") if os.path.exists("/content/drive/My Drive") else "No My Drive")


['.shortcut-targets-by-id', 'MyDrive', '.Trash-0', '.Encrypted']
['Can you create a graph that contains all the sign....gsheet', 'Budget Table.gsheet', 'Reana: SEO and AI Optimization\n.gdoc', 'Expense_Splitter_Template.xlsx', 'Untitled spreadsheet.gsheet', 'Japan.gsheet', 'Colab Notebooks', 'extracted_folder']
['Can you create a graph that contains all the sign....gsheet', 'Budget Table.gsheet', 'Reana: SEO and AI Optimization\n.gdoc', 'Expense_Splitter_Template.xlsx', 'Untitled spreadsheet.gsheet', 'Japan.gsheet', 'Colab Notebooks', 'extracted_folder']


In [10]:
import os
import pandas as pd

OUT_DIR = "/content/drive/MyDrive/irvine_recommender_outputs"

parquet_files = {
    "oc_business": os.path.join(OUT_DIR, "oc_business.parquet"),
    "oc_review":   os.path.join(OUT_DIR, "oc_review.parquet"),
    "oc_user":     os.path.join(OUT_DIR, "oc_user.parquet"),
}

for name, pqt_path in parquet_files.items():
    if not os.path.exists(pqt_path):
        print(f"❌ Missing: {pqt_path}")
        continue

    print(f"Reading {pqt_path} ...")
    df = pd.read_parquet(pqt_path)

    csv_path = os.path.join(OUT_DIR, f"{name}.csv")
    df.to_csv(csv_path, index=False)

    print(f"✅ Wrote {csv_path}  |  rows={len(df):,}, cols={df.shape[1]}")

print("\nDone. Check your Drive folder:", OUT_DIR)


Reading /content/drive/MyDrive/irvine_recommender_outputs/oc_business.parquet ...
✅ Wrote /content/drive/MyDrive/irvine_recommender_outputs/oc_business.csv  |  rows=1,349, cols=14
Reading /content/drive/MyDrive/irvine_recommender_outputs/oc_review.parquet ...
✅ Wrote /content/drive/MyDrive/irvine_recommender_outputs/oc_review.csv  |  rows=225,242, cols=11
Reading /content/drive/MyDrive/irvine_recommender_outputs/oc_user.parquet ...
✅ Wrote /content/drive/MyDrive/irvine_recommender_outputs/oc_user.csv  |  rows=109,354, cols=22

Done. Check your Drive folder: /content/drive/MyDrive/irvine_recommender_outputs
